In [1]:
import os

from sleep_events import connect_to_firebase

firebase = connect_to_firebase()

In [2]:
import pandas as pd

docs = firebase.collection('notesExperimental').stream()

note_records = [{'id': doc.id, **doc.to_dict()} for doc in docs]

note_df = pd.DataFrame(note_records)

In [3]:
note_df[note_df['dayAndNightOf'].isnull()]

,id,timestampForUTC,timestampWrittenUTC,timestampFor,tags,dayAndNightOf,timestampWritten,note,timestampUpdated,isPreviousDay


In [4]:
firebase_grouped = note_df.groupby('dayAndNightOf').size()
firebase_grouped

dayAndNightOf
2024-05-03    2
2024-05-04    1
2024-05-05    8
2024-05-06    7
2024-05-07    8
             ..
2025-01-27    5
2025-01-28    9
2025-01-29    7
2025-01-30    9
2025-01-31    2
Length: 272, dtype: int64

In [4]:
note_df[note_df['dayAndNightOf'] == '2024-07-11']

,id,note,dayAndNightOf,timestampForUTC,timestampWritten,timestampFor,tags,timestampWrittenUTC,timestampUpdated,isPreviousDay


In [5]:
import meilisearch
import json
import json
import re
import os
import dotenv
from pymongo import MongoClient

dotenv.load_dotenv("../../.env")

client = MongoClient(os.getenv('MONGO_URI'))
db = client['examined_life']
collection = db['notes']


In [13]:
unique_tags = collection.aggregate([
    {
        "$unwind": "$tags"
    },
    {
        "$group": {
            "_id": None,
            "uniqueTags": { "$addToSet": "$tags" }
        }
    },
    {
        "$project": {
            "_id": 0,
            "uniqueTags": 1
        }
    }
])

# Convert the result to a list and extract the unique tags
unique_tags_list = list(unique_tags)[0]['uniqueTags']
unique_tags_list

['settle:early',
 'elvanse:discomfort',
 'melatonin:insomnia',
 'sleep:disturbance:sunlight',
 'settling:fired-up',
 'sleep:histamine',
 'settling:internet',
 'pillow:levitex',
 'energy:food:perking',
 'fast:concerta',
 'concerta:27mg:next-day',
 'ritalin:emotional',
 'working-late',
 'ritalin:doing-without',
 'sleep:late-night',
 'ritalin:mylan:5mg-enough',
 'energy:slumps:post-morning-walk',
 'concerta:despite-poor-sleep',
 'pillow:wrinkles',
 'concerta:kicking-in',
 'energy:improved-by-large-meal',
 'settling:dodow',
 'concerta:insomnia',
 'settling:rgm',
 'ritalin:kicking-in',
 'morning-light',
 'sleep:weed',
 'circadian-rhythm:daylight-savings',
 'adhd:protein',
 'sleep:blackouts',
 'settle:audiobooks',
 'hayfever',
 'sleep:disturbance:cold',
 'mystery',
 'ruminations',
 'circadian-rhythm:summary',
 'reasons-to-leave-couchbase',
 'settling:morning-light',
 'settling:games',
 'elvanse:10mg',
 'elvanse:productivity',
 'circadian-rhythm:alarm',
 'experiment:morning-walk',
 'productiv

In [19]:
all_docs = list(collection.find({}))

In [27]:
import numpy as np
[doc for doc in all_docs if doc['dayAndNightOf'] == np.nan]

[]

In [30]:
import pandas as pd

# Extract headers from the first document
headers = all_docs[0].keys()

# Convert the list of documents to a DataFrame using the extracted headers
all_docs_df = pd.DataFrame(all_docs, columns=headers)

# Display the DataFrame
all_docs_ml_grouped = all_docs_df.groupby('dayAndNightOf').size()
all_docs_ml_grouped

dayAndNightOf
2024-05-03    2
2024-05-04    1
2024-05-05    8
2024-05-06    7
2024-05-07    8
             ..
2025-01-27    5
2025-01-28    9
2025-01-29    7
2025-01-30    9
2025-01-31    2
Length: 272, dtype: int64

In [31]:
firebase_grouped

dayAndNightOf
2024-05-03    2
2024-05-04    1
2024-05-05    8
2024-05-06    7
2024-05-07    8
             ..
2025-01-27    5
2025-01-28    9
2025-01-29    7
2025-01-30    9
2025-01-31    2
Length: 272, dtype: int64

In [32]:
comparison = firebase_grouped.compare(all_docs_ml_grouped)
comparison

,self,other
dayAndNightOf,,


In [33]:
all_docs_df[all_docs_df['dayAndNightOf'] == '2024-07-12']

,_id,timestampFor,tags,dayAndNightOf,note,timestampWritten
309,2024-07-12T10:21:13.254+01:00,2024-07-12 09:21:13,"[sleep:summary, sleep:temp]",2024-07-12,Am feeling a lot better this morning.\nGot the...,2024-07-12 09:21:13.254
310,2024-07-12T14:36:12.049+01:00,2024-07-12 13:36:12,"[co2, sleep:summary, energy:good, sleep:good, ...",2024-07-12,Today has been pretty good until now when I'm ...,2024-07-12 13:36:12.049
311,2024-07-13T13:15:33.200+01:00,NaT,[co2],2024-07-12,Night of 12th: Co2 change: pumped air out alon...,2024-07-14 15:35:20.672


In [36]:
filter_query = {
    "dayAndNightOf": {
        "$gte": '2024-07-12',
        "$lte": '2024-07-12'
    }
}

# Execute the query
results = collection.find(filter_query).limit(1000).skip(0)

# Convert the results to a list
results_list = list(results)
results_list

[{'_id': '2024-07-12T10:21:13.254+01:00',
  'tags': ['sleep:summary', 'sleep:temp'],
  'dayAndNightOf': '2024-07-12',
  'timestampWritten': datetime.datetime(2024, 7, 12, 9, 21, 13, 254000),
  'timestampFor': datetime.datetime(2024, 7, 12, 9, 21, 13),
  'note': 'Am feeling a lot better this morning.\nGot the bedroom nice and cool yday.'},
 {'_id': '2024-07-12T14:36:12.049+01:00',
  'timestampWritten': datetime.datetime(2024, 7, 12, 13, 36, 12, 49000),
  'tags': ['co2',
   'sleep:summary',
   'energy:good',
   'sleep:good',
   'settling:audiobooks',
   'sleep:disturbance:temperature',
   'sleep:blackouts'],
  'dayAndNightOf': '2024-07-12',
  'timestampFor': datetime.datetime(2024, 7, 12, 13, 36, 12),
  'note': "Today has been pretty good until now when I'm slumping after sandwich lunch.  Yday:\nDinner was brown rice + chicken and veg, 7 to 8 ish.\nWorked on XL until 9.  Watched Clarksons Farm, upstairs about 10.  Had a shower.  Was in bed at 10.45 after sorting out EEG montage, fans, wi

In [38]:
results = list(collection.find({"tags": {"$regex": "ritalin", "$options": "i"}}))
[r['tags'] for r in results]

[['ritalin', 'inhibition', 'autistic'],
 ['poor-settling-leads-to-poor-sleep', 'sleep', 'ritalin'],
 ['ritalin'],
 ['adhd', 'ritalin'],
 ['ritalin', 'productivity'],
 ['adhd', 'ritalin', 'work-productivity'],
 ['adhd', 'ritalin', 'sleep', 'energy:slumps'],
 ['ritalin', 'energy:afternoon-slump'],
 ['settle', 'ritalin'],
 ['ritalin', 'work:blocked', 'energy:slumps'],
 ['settling:games', 'sleep:blackouts', 'ritalin'],
 ['ritalin', 'adhd:protein'],
 ['ritalin', 'ritalin:20mg'],
 ['ritalin', 'energy:slumps'],
 ['ritalin'],
 ['ritalin', 'adhd'],
 ['ritalin', 'drinking', 'energy:slumps'],
 ['ritalin', 'energy:afternoon-slump', 'adhd'],
 ['ritalin', 'work-productivity', 'energy:slumps'],
 ['ritalin', 'sleep:cold-feet', 'ritalin:affecting-sleep'],
 ['ritalin'],
 ['ritalin', 'ritalin:10mg-too-much'],
 ['ritalin', 'ritalin:5mg-too-much', 'ritalin:5mg-not-enough'],
 ['ritalin', 'ritalin:5mg-too-much', 'ritalin:5mg-not-enough'],
 ['ritalin'],
 ['ritalin', 'settling'],
 ['ritalin', 'hayfever'],
 ['r

In [42]:
list(collection.find({
    "$or": [
        {"tags": {"$regex": "ritalin", "$options": "i"}},
        {"note": {"$regex": "ritalin", "$options": "i"}},
        {"note": {"$regex": "rit", "$options": "i"}}
    ]
}))

[{'_id': '2024-05-03T16:48:08.446+01:00',
  'timestampFor': datetime.datetime(2024, 5, 3, 15, 47, 17, 610000),
  'tags': ['ritalin', 'inhibition', 'autistic'],
  'dayAndNightOf': '2024-05-03',
  'note': "Interesting ritalin thing.. It suppresses my inhibition enough to make me respond quicker, and that's really making me realise that I am a bit autistic. Maybe?",
  'timestampWritten': datetime.datetime(2024, 5, 3, 15, 48, 8, 446000)},
 {'_id': '2024-05-03T22:16:32.279+01:00',
  'timestampFor': datetime.datetime(2024, 5, 3, 21, 14, 39, 301000),
  'tags': ['poor-settling-leads-to-poor-sleep', 'sleep', 'ritalin'],
  'note': "Last night stayed up on laptop until last minute.  Was too stimulated.  Then woke up early.  Pretty clear there's a connection between how stimulated I am during settle, and how early I wake.\nWas then absolutely knackered today, until ritalin saved me.",
  'timestampWritten': datetime.datetime(2024, 5, 3, 21, 16, 32, 280000),
  'dayAndNightOf': '2024-05-03'},
 {'_id'